In [20]:
import h5py as h5
import numpy as np

In [2]:
# Modifying sys.path to be able to load project packages
import sys
sys.path.append('../')

# Load project packages
from utils.h5_tools import write_batch

In [13]:
path_1 = 'C:/data/datasets/sh_bh_datasets/2014_2019_global_norm_bh_sh.h5'
path_2 = 'C:/data/false_positives/data_converted.h5'

In [39]:
# label: maximum data to replace
max_merge_ratio = {0: 0., 1: 0., 2: 0.5}

In [15]:
save_path = 'C:/data/false_positives/2014_2019_with_false_positives.h5'

In [16]:
d_set_1 = h5.File(path_1, 'r')
d_set_2 = h5.File(path_2, 'r')

In [17]:
d_set_1.keys(), d_set_2.keys()

(<KeysViewHDF5 ['ID', 'X', 'Y']>, <KeysViewHDF5 ['X', 'Y', 'Z']>)

In [18]:
# set_2 key: set_2 key
keys_transistion_2_1 = {'X': 'X', 'Y': 'Y', 'Z': 'ID'}

# label key for set_2
label_key_2 = 'Y'

In [59]:
data_1 = {}
data_2 = {}

for key_2, key_1 in keys_transistion_2_1.items():
    
    data_1[key_1] = np.array(d_set_1[key_1])
    data_2[key_2] = np.array(d_set_2[key_2])

In [60]:
labels_1 = data_1[keys_transistion_2_1[label_key_2]]
labels_2 = data_2[label_key_2]

In [61]:
# Label count and merge data count
label_count = {}
max_merge_values = {}

for label, ratio in max_merge_ratio.items():
    
    count = (labels_1 == label).sum()
    count_2 = (labels_2 == label).sum()
    
    max_merge = min(int(count * ratio), count_2)
    
    label_count[label] = count
    max_merge_values[label] = max_merge

In [62]:
label_count, max_merge_values

({0: 2630, 1: 3185, 2: 2630}, {0: 0, 1: 0, 2: 1315})

In [50]:
def shuffle_in_unison(*arrays):
    
    assert len(set([arr.shape[0] for arr in arrays])) == 1
    
    shuffled = [np.empty(arr.shape, dtype = arr.dtype) for arr in arrays]
    
    permutation = np.random.permutation(shuffled[0].shape[0])
    
    for old_index, new_index in enumerate(permutation):
        for arr_shufled, arr in zip(shuffled, arrays):
            arr_shufled[new_index] = arr[old_index]
        
    return shuffled

In [65]:
def shuffle_set(set_dictionary):
    
    labels = []
    data = []
    
    for l, d in set_dictionary.items():
        labels.append(l)
        data.append(d)
        
    data = shuffle_in_unison(*data)
    
    s_set_dictionary = {}
    for l, d in zip(labels, data):
        s_set_dictionary[l] = d
        
    return s_set_dictionary

In [66]:
data_1 = shuffle_set(data_1)
data_2 = shuffle_set(data_2)

In [69]:
max_merge_values

{0: 0, 1: 0, 2: 1315}

In [104]:
# Merge
labels_1 = data_1[keys_transistion_2_1[label_key_2]]
labels_2 = data_2[label_key_2]

length = labels_1.shape[0]

for label, n_merge in max_merge_values.items():
    
    if not n_merge:
        continue
        
    count = 0
    for i in range(length):
        
        if count == n_merge:
            break
            
        if labels_1[i] != label:
            continue
            
        for key_2, key_1 in keys_transistion_2_1.items():
            
            data_1[key_1][i] = data_2[key_2][i]
            
        count += 1

In [107]:
# Save dataset
save_path 

'C:/data/false_positives/2014_2019_with_false_positives.h5'

In [109]:
string_keys = ['Z', 'ID']

In [118]:
for key, data in data_1.items():
    
    if key in string_keys:
        write_batch(save_path, key, data, string = True)
    else:
        write_batch(save_path, key, data)

## Check saved set

In [119]:
import matplotlib.pyplot as plt

In [120]:
f = h5.File(save_path, 'r')

In [121]:
f.keys()

<KeysViewHDF5 ['ID', 'X', 'Y']>

In [122]:
f['X']

<HDF5 dataset "X": shape (8445, 400, 3), type "<f8">

In [126]:
f['Y'][:12], f['ID'][:12]

(array([0., 0., 2., 2., 1., 1., 0., 1., 0., 0., 2., 1.]),
 array(['20160904171245_l10_sARGI', '20170303075954_l8_sA759',
        'false_positive', 'false_positive', '20180621153307_l16_sARGI',
        '20191223100221_l17_sARGI1', '20140701175226_l10_sARGI',
        '20190926004233_l17_sVAL1', '20170210095100_l18_sNGLK',
        '20160816030014_l17_sARGI', 'false_positive',
        '20150506184627_l16_sNYSH'], dtype=object))